In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
!gdown --id 1s2ggWGqAjB-wlXkd6Pu7pM_3kqs6RhAF
!gdown --id 1iQCM32OzxqvmDsgL5j4et-yE5W8o6afE
!gdown --id 1bs8PHTExPfItW636-HVRVYwjjPilQVgy

Mounted at /content/gdrive/
Downloading...
From: https://drive.google.com/uc?id=1s2ggWGqAjB-wlXkd6Pu7pM_3kqs6RhAF
To: /content/id_map.parquet
100% 1.20M/1.20M [00:00<00:00, 79.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1iQCM32OzxqvmDsgL5j4et-yE5W8o6afE
To: /content/train.csv
42.9MB [00:01, 24.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bs8PHTExPfItW636-HVRVYwjjPilQVgy
To: /content/test.csv
24.8MB [00:00, 53.4MB/s]


In [ ]:
!pip install scikit-learn==0.24
!pip install tldextract
!pip install eli5
!pip install category_encoders
!pip install hyperopt

     |████████████████████████████████| 22.3 MB 2.0 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 87 kB 2.7 MB/s 
     |████████████████████████████████| 106 kB 4.1 MB/s 
     |████████████████████████████████| 80 kB 2.9 MB/s 


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy import stats
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from functools import partial

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

import time
from tqdm import tqdm_notebook

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OrdinalEncoder

import re
from wordcloud import WordCloud
from tldextract import extract
from gensim.models import Word2Vec

import warnings
warnings.filterwarnings("ignore")

import six
from pandas.api.types import is_sparse
from category_encoders import TargetEncoder

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, TimeSeriesSplit
from numpy import mean
from numpy import std
from scipy.sparse import hstack
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Support

In [ ]:
def time_feature_extraction(data, train_flag):

    data = data.assign(
        # session start time
        session_start=lambda x: x.filter(like='time').min(axis=1),
        # session end time
        session_end=lambda x: x.filter(like='time').max(axis=1),
        # session duration
        duration=lambda x: (x.session_end - x.session_start).dt.seconds,
        # start hour in a session
        start_hour=lambda x: x.session_start.apply(lambda x: x.hour).astype('int'),
        # day of week in a session
        weekday=lambda x: x.session_start.apply(lambda x: x.dayofweek).astype('category'),

        # count_of_nans
        nans_count=lambda x: x.filter(like='webpage').isna().sum(axis=1),
        # nunique pages
        nunique_pages=lambda x: x.filter(like='webpage').apply(lambda row: row.nunique(), axis=1) / (10 - x.nans_count)
        )

    data.loc[data["duration"]==0, "duration"] = 0.001
    data["duration"], lmbda = stats.boxcox(data.loc[:, "duration"])

    data.loc[data["start_hour"].isin([15,16,17,18,19,3]), "start_hour"] = 17

    if train_flag:
        data = data.reset_index().sort_values(['session_start', "session_id"], ignore_index=True)
        data.drop(columns=["session_start", "session_end"], inplace=True)
    else:
        data.drop(columns=["session_start", "session_end"], inplace=True)
    return data

In [ ]:
def intersection(train, id_map):
    "seq of webpages"
    data_train = merge_data_and_domain(train, id_map).loc[:, ['webpage_update%s' % i for i in range(1, 11)]+["start_hour", "weekday"]]

    data_train.loc[:, ['webpage_update%s' % i for i in range(1, 11)]] = data_train.loc[:, ['webpage_update%s' % i for i in range(1, 11)]].fillna("")
    data_train.loc[:, "string"] = data_train.apply(lambda x: "-".join([x[i] for i in range(0,10) if x[i] != ""]), axis=1)
    data_train.loc[:, "preproc_tokens"] = data_train.loc[:, "string"].apply(lambda x: x.split('-')).values

    data_train = data_train.loc[:, ["preproc_tokens"]]

    data_train.loc[:, [".youtube", "sn gxouxg jqbe.googlevideo", ".ytimg"]] = 0

    for token in [".youtube", "sn gxouxg jqbe.googlevideo", ".ytimg"]:
        data_train[token] = data_train.loc[:, "preproc_tokens"].apply(lambda x: 1 if token in x else 0)
    data_train["intersection"] = 0
    data_train.loc[(data_train[".youtube"]==1) & (data_train["sn gxouxg jqbe.googlevideo"]==1) & (data_train[".ytimg"]==1), "intersection"] = 1

    train = pd.concat([train, data_train.loc[:, ["intersection"]]], axis=1)

    return train

In [ ]:
def text_preprocessing_1(text):
    """text_preproc"""
    # split text by . or -
    clean_text = re.split("\.|-", text)
    # join text
    clean_text = " ".join(clean_text)
    # remove nums
    clean_text = re.sub("[0-9]", "", clean_text)
    # remove spaces
    clean_text = re.sub(' +', ' ', clean_text)
    # split
    clean_text = re.split(" ", clean_text)
    # seq must be longer then 1
    clean_text = [x for x in clean_text if len(x)>1]
    # join text
    clean_text = " ".join(clean_text)

    return clean_text

In [ ]:
def merge_data_and_domain(data, id_map):
    """merge sub_domain + domain to full webapage"""
    for i in range(1,11):
        data = pd.merge(data, id_map.rename(columns={"id":"webpage"+str(i), 
                                                     "webpage_update":"webpage_update"+str(i)}).loc[:, ["webpage"+str(i),
                                                                                                        "webpage_update"+str(i)]], 
                        on="webpage"+str(i), how="left")
    return data
    
def tf_idf_features(train, test, vectorizer_params):
    """apply tf-idf to sub_domain + domain"""

    webpages_update = ['webpage_update%s' % i for i in range(1, 11)]

    train_sessions = train.loc[:, webpages_update].fillna("")
    test_sessions = test.loc[:, webpages_update].fillna("")

    train_tokens = train_sessions.apply(lambda x: "-".join([x[i] for i in range(0,10) if x[i] != ""]), axis=1)
    test_tokens = test_sessions.apply(lambda x: "-".join([x[i] for i in range(0,10) if x[i] != ""]), axis=1)

    vectorizer = TfidfVectorizer(**vectorizer_params)
    X_train = vectorizer.fit_transform(train_tokens)
    X_test = vectorizer.transform(test_tokens)
    
    return X_train, X_test, vectorizer

In [ ]:
def sparse_to_dense(x_train, x_test):
    """sparse matrix to normal dense"""
    for column in x_train.columns:
        if is_sparse(x_train[column]):
            x_train.loc[:, column] = x_train.loc[:, column].sparse.to_dense()

    for column in x_test.columns:
        if is_sparse(x_test[column]):
            x_test.loc[:, column] = x_test.loc[:, column].sparse.to_dense()
    return x_train, x_test

In [ ]:
def extraction(train, x_test, id_map, vectorizer_params, 
               time_features, popular_features, 
               columns_tf_idf, tf_idf, ohe_flag):

    if time_features:      
        print("extract time features")    
        train = time_feature_extraction(train, train_flag=True)
        x_test = time_feature_extraction(x_test, train_flag=False)

    if popular_features:
        print("intersection features") 
        train = intersection(train.copy(), id_map.copy())
        x_test = intersection(x_test, id_map)

    if tf_idf:
        print("apply tf idf")  
        data_train = merge_data_and_domain(train, id_map).loc[:, ['webpage_update%s' % i for i in range(1, 11)]]
        data_test = merge_data_and_domain(x_test, id_map).loc[:, ['webpage_update%s' % i for i in range(1, 11)]]
        train_tf_idf, test_tf_idf, vectorizer = tf_idf_features(train=data_train, test=data_test, vectorizer_params=vectorizer_params)
        train_tf_idf = pd.DataFrame.sparse.from_spmatrix(train_tf_idf, 
                                                         columns=vectorizer.get_feature_names(), 
                                                         index=train.set_index("session_id").index)
        train_tf_idf = train_tf_idf.loc[:, columns_tf_idf]
        test_tf_idf = pd.DataFrame.sparse.from_spmatrix(test_tf_idf, 
                                                        columns=vectorizer.get_feature_names(), 
                                                        index=x_test.set_index("session_id").index)
        test_tf_idf = test_tf_idf.loc[:, columns_tf_idf]

    # set_index
    train.set_index("session_id", inplace=True)
    x_test.set_index("session_id", inplace=True)

    x_train, y_train = train.drop(columns=["target"]), train.loc[:, ["target"]]

    # drop useless features
    other_features_list = ["time"+str(i) for i in range(1,11)]
    x_train.drop(columns=other_features_list, inplace=True)
    x_test.drop(columns=other_features_list, inplace=True)

    if ohe_flag:
        encoder = OneHotEncoder(handle_unknown="ignore")
        features_dummy_train = encoder.fit_transform(x_train.loc[:, ["weekday"]])
        features_dummy_train = pd.DataFrame.sparse.from_spmatrix(features_dummy_train, index=x_train.index)
        features_dummy_test = encoder.transform(x_test.loc[:, ["weekday"]])
        features_dummy_test = pd.DataFrame.sparse.from_spmatrix(features_dummy_test, index=x_test.index)

        cat_columns = list(range(features_dummy_test.shape[1]))

        x_train.drop(columns=["weekday"], inplace=True)
        x_test.drop(columns=["weekday"], inplace=True)

        x_train = pd.concat([features_dummy_train, x_train], axis=1)
        x_test = pd.concat([features_dummy_test, x_test], axis=1)

    columns = x_train.columns

    if tf_idf:
        x_train = pd.concat([x_train, train_tf_idf], axis=1)
        x_test = pd.concat([x_test, test_tf_idf], axis=1)
        x_train, x_test = sparse_to_dense(x_train, x_test)
        return x_train, y_train, x_test, vectorizer, columns, cat_columns
    else:
        x_train, x_test = sparse_to_dense(x_train, x_test)
        return x_train, y_train, x_test, columns, cat_columns

In [ ]:
def save_submission(pred, number):
    pd.Series(
        pred, name='target', index=pd.Index(range(len(pred)), name='session_id')
    ).to_csv('/content/gdrive/MyDrive/EPAM/Week 7. Trees/HW/submissions_test/notebook_submission' + str(number) + '.csv')

In [ ]:
def objective(space, estimator, x_train, y_train):
    time_split = TimeSeriesSplit(n_splits=7)

    params = {
        'learning_rate': space['learning_rate'],
        'reg_alpha': space['reg_alpha'],
        'reg_lambda': space['reg_lambda'],
        'num_leaves': int(space['num_leaves']),
        'n_estimators': int(space['n_estimators']),
        'min_child_samples': int(space['min_child_samples']),
    }
    
    # задаём модели требуемые параметры    
    estimator.set_params(**params)
    
    score = cross_val_score(estimator, x_train, y_train, scoring='roc_auc', cv=time_split).mean()
    print("AUC {:.3f} params {}".format(score, params))
    return {'loss':1 - score, 'status': STATUS_OK }

In [ ]:
def df_results(hp_results):
    """
    Отображаем результаты hyperopt в формате DataFrame 

    :hp_results: результаты hyperop
    :return: pandas DataFrame
    """ 

    results = pd.DataFrame([{**x, **x['params']} for x in  hp_results])
    results.drop(labels=['status', 'params'], axis=1, inplace=True)
    results.sort_values(by=['loss'], ascending=False, inplace=True)
    return results

# Preprocessing

In [ ]:
# usefull features tf-idf
list_positive = ['.express', '.info jeunes', '.vk', '.melty', 'fr web img.acsta',
                '.videostep', 'khms.google', '.radio canada', 'api.bing',
                '.banque chalus', '.audienceinsights', 'demotivateur.disqus',
                '.indeed', 'media.melty', '.video', '.jobisjob', '.blogger',
                'gujynbgx.admedia', '.youwatch', 'reviewer.lavoixdunord',
                'img wikia.nocookie', 'static.programme tv', 'static.flickr',
                '.audienceinsights facebook', '.regarder film gratuit', '.bbc',
                'facebook .audienceinsights', 'static.videostep', '.exashare',
                '.blastr', 'docs.google', 'graphics.nytimes', '.reddit']

list_neg = ['.futura sciences', 'clients.google clients.google clients.google',
            'fr mg mail.yahoo', '.ztat', 'fbstatic.akamaihd', '.spin',
            'facebook', 'ba.commentcamarche', 'static.ccm', 'safebrowsing cache.google', 
            '.linkedin', '.wordreference', 'fr.openclassrooms', '.bing', 
            'clients.google clients.google', 'mail.google', 'plus.google']

features_list = list_positive + list_neg

In [ ]:
OUTPUT_FOLDER = "/content/gdrive/MyDrive/EPAM/Week 7. Trees/HW/model_test"

In [ ]:
train = pd.read_csv("./train.csv", index_col=0, parse_dates=[f'time{i+1}' for i in range(10)])
x_test = pd.read_csv('./test.csv', index_col=0, parse_dates=[f'time{i+1}' for i in range(10)])
x_test.reset_index(inplace=True)
id_map = pd.read_parquet("/content/id_map.parquet")

In [ ]:
# get user's webpages 
user_frame = train.loc[train["target"]==1].reset_index(drop=True)

array = np.array([])
for element in ["webpage"+str(x) for x in range(1,11)]:
    unique_array = user_frame.loc[:, element].unique()
    array = np.concatenate([array, unique_array])
array = {x for x in array if x==x}

id_map["target"] = 0
id_map.loc[id_map["id"].isin(array), "target"] = 1

# extract "sub-domain", "domain", "suf" from url of webpage
list_ = [list(extract(id_map.loc[i].webpage)) for i in id_map.index]
id_map.loc[:, ["sub","domain","suf"]] = list_

# url preprocessing
id_map.loc[:, "domain"] = [text_preprocessing_1(text=x) for x in id_map.loc[:, "domain"].values]
id_map.loc[:, "sub"] = [text_preprocessing_1(text=x) for x in id_map.loc[:, "sub"].values]
id_map.loc[:, "sub"] = id_map.loc[:, "sub"].apply(lambda x: "" if x in ["www", "js init"] else x)
id_map["webpage_update"] = id_map.apply(lambda x: ".".join([x[3], x[4]]), axis=1)
id_map.loc[:, "webpage_update"] = id_map.loc[:, "webpage_update"].apply(lambda x: "twitter" if x in [".twitter", "platform.twitter"] else x)
id_map.loc[:, "webpage_update"] = id_map.loc[:, "webpage_update"].apply(lambda x: "facebook" if x in ["static ak.facebook", ".facebook", "connect.facebook"] else x)

In [ ]:
# params for tf-idf
vectorizer_params={'ngram_range': (1, 5), 'max_features': 30000,
                   "min_df": 2, "max_df": 0.7,
                   'tokenizer': lambda s: s.split("-")
                   }

In [ ]:
# time_features + popular_features
x_train3, y_train3, x_test3, vectorizer3, columns3, cat_columns3 = extraction(train=train.copy(), x_test=x_test.copy(), id_map=id_map, 
                                                                   vectorizer_params=vectorizer_params, time_features=True,
                                                                   popular_features=True, columns_tf_idf=features_list,
                                                                   tf_idf=True, ohe_flag=True)

x_train3.fillna(0, inplace=True)
y_train3.fillna(0, inplace=True)

extract time features
intersection features
apply tf idf


## LGBMClassifier

In [ ]:
estimator = LGBMClassifier(random_state=123, is_unbalance=True)

space = {
    'learning_rate': hp.loguniform('learning_rate', low=-4*np.log(10), high=-1*np.log(10)),
    'reg_alpha': hp.loguniform('reg_alpha', -4*np.log(10), 2*np.log(10)),
    'reg_lambda': hp.loguniform('reg_lambda', -4*np.log(10), 2*np.log(10)),
    'num_leaves': hp.uniform('num_leaves', 10, 80),
    'n_estimators': hp.uniform('n_estimators', 100, 1000),
    'min_child_samples': hp.uniform('min_child_samples', 10, 200),
    }

trials = Trials()

best = fmin(
    # функция для оптимизации 
    fn=partial(objective, estimator=estimator, x_train=x_train1, y_train=y_train1),
    # пространство поиска гиперпараметров
    space=space,
    # алгоритм поиска
    algo=tpe.suggest,
    # число итераций
    max_evals=450,
    # куда сохранять историю поиска
    trials=trials,
    # random state
    rstate=np.random.RandomState(1),
    # progressbar
    show_progressbar=True
    )

In [ ]:
time_split = TimeSeriesSplit(n_splits=7)

estimator_3 = LGBMClassifier(boosting_type="goss",  class_weight='balanced',
                             min_child_samples=115, n_estimators=400, num_leaves=32,
                             random_state=42, reg_alpha=0.25, reg_lambda=0.25)

score = cross_val_score(estimator_3, x_train3, y_train3, scoring='roc_auc', cv=time_split).mean()
estimator_3.fit(x_train3, y_train3)
print("score: ", score)

score:  0.9145039288638277


In [ ]:
# predict
lgbm_pred = estimator_3.predict_proba(x_test3)[:, 1]

In [ ]:
# save Submission
save_submission(lgbm_pred, number=47)